# Land price web scraping project


## Steps involved
- Import libraries
- Create ETL functions
- Scrape the data
- Create CSV file of the data

### 1.) Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup